In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/AD Identification using SATD'

Mounted at /content/drive
/content/drive/My Drive/AD Identification using SATD


#Import basic libraries

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import LabelEncoder


from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder

from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC



#Read the datasets

In [ ]:
import pandas as pd
liu_ = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
liu_ = pd.read_csv(liu_, low_memory=False)

In [ ]:
liu_['Comments'].fillna('', inplace=True)
liu_['TDType'] = liu_['TDType'].astype(str)

# Replace values with 'WITHOUT CLASSIFICATION'
values_to_remove = ['MULTITHREAD', 'nan', 'removeType']
replacement_value = 'WITHOUT_CLASSIFICATION'
liu_['TDType'].replace(values_to_remove, replacement_value, inplace=True)

liu_['Comments'] = liu_['Comments'].str.replace('content=', '', regex=False)
liu_['Comments'] = liu_['Comments'].str.replace('"', '', regex=False)

In [ ]:
# Remove duplicates
liu_ = liu_.drop_duplicates(subset=['Comments', 'TDType'])
liu_['TDType'] = liu_['TDType'].replace('removeType', 'WITHOUT_CLASSIFICATION')


In [ ]:
# Count the number of duplicate rows in the DataFrame
num_duplicates = liu_.duplicated().sum()

print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 0


In [ ]:
pip install transformers


In [ ]:
import wandb


wandb.init(mode="disabled")

##RoBERTa Model

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np

data = liu_

# Map TDType classes to integers for training
class_mapping = {label: idx for idx, label in enumerate(data['TDType'].unique())}
data['label'] = data['TDType'].map(class_mapping)

# Split the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(data['Comments'], data['label'], test_size=0.2, random_state=42, stratify=data['label'])
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp)

# Set up tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(class_mapping))


class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = self.labels.iloc[idx]

        # Tokenize the text
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_dataset = CustomDataset(X_train, y_train, tokenizer)
val_dataset = CustomDataset(X_val, y_val, tokenizer)
test_dataset = CustomDataset(X_test, y_test, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

# Define metrics function for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1_score(labels, preds, average='weighted')
    }

# Initialise the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


trainer.train()

# Evaluate the model on the test set
test_results = trainer.predict(test_dataset)

# Classification report
y_test_pred = np.argmax(test_results.predictions, axis=1)
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_test_pred, target_names=class_mapping.keys()))

# Extract and save embeddings
def extract_embeddings(model, dataset):
    embeddings = []
    dataloader = DataLoader(dataset, batch_size=16)

    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            inputs = {
                'input_ids': batch['input_ids'].to(model.device),
                'attention_mask': batch['attention_mask'].to(model.device),
            }
            outputs = model.roberta(**inputs)
            hidden_states = outputs.last_hidden_state[:, 0, :]  # Extract [CLS] token embeddings
            embeddings.append(hidden_states.cpu().numpy())

    return np.concatenate(embeddings, axis=0)

# Extract embeddings for train, val, and test sets
embeddings_train = extract_embeddings(model, train_dataset)
embeddings_val = extract_embeddings(model, val_dataset)
embeddings_test = extract_embeddings(model, test_dataset)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.442400,0.446496,0.857235,0.839901
2,0.472500,0.385176,0.881833,0.869646
3,0.225700,0.450344,0.886334,0.885922
4,0.312900,0.402543,0.885531,0.886154
5,0.203100,0.413448,0.900643,0.897568
6,0.141100,0.559913,0.895177,0.892383
7,0.125200,0.571976,0.894855,0.897219
8,0.109600,0.591428,0.900804,0.898161
9,0.121300,0.544915,0.910289,0.908199
10,0.141500,0.591264,0.904984,0.903882



Classification Report on Test Set:
                        precision    recall  f1-score   support

             ALGORITHM       0.45      0.45      0.45        38
         COMPATIBILITY       0.82      0.50      0.62        18
                DEFECT       0.62      0.58      0.60        26
                DESIGN       0.84      0.89      0.87       436
         DOCUMENTATION       0.50      0.67      0.57         6
        IMPLEMENTATION       0.83      0.70      0.76        79
                  TEST       0.83      0.73      0.78        26
WITHOUT_CLASSIFICATION       0.97      0.97      0.97       926

              accuracy                           0.90      1555
             macro avg       0.73      0.68      0.70      1555
          weighted avg       0.90      0.90      0.90      1555



##Save Embeddings

In [ ]:
# Save embeddings to CSV files
def save_embeddings_to_csv(embeddings, file_path, labels=None):

    df = pd.DataFrame(embeddings)
    if labels is not None:
        df['label'] = labels
    df.to_csv(file_path, index=False)

# Save embeddings with corresponding labels
save_embeddings_to_csv(embeddings_train, 'train_embeddings_.csv', y_train.values)
save_embeddings_to_csv(embeddings_val, 'val_embeddings_.csv', y_val.values)
save_embeddings_to_csv(embeddings_test, 'test_embeddings_.csv', y_test.values)

print("Embeddings saved to CSV files.")


Embeddings saved to CSV files.


##RobERTA Embeddings

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# Load embeddings
train = pd.read_csv('train_embeddings_.csv')
val = pd.read_csv('val_embeddings_.csv')
test = pd.read_csv('test_embeddings_.csv')

X_train = train.iloc[:, :-1]  # All columns except the last (features)
y_train = train['label']      # Last column as label

X_val = val.iloc[:, :-1]
y_val = val['label']

X_test = test.iloc[:, :-1]
y_test = test['label']

# Scale training and validation data separately
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


lr_model = LogisticRegression(max_iter=1000, random_state=42, solver='liblinear', penalty='l2', C=10, class_weight='balanced')
lr_model.fit(X_train_scaled, y_train)

# Evaluate on the validation set
y_val_pred = lr_model.predict(X_val_scaled)


# Predict on the test set
y_test_pred = lr_model.predict(X_test_scaled)

# Evaluate the model on the test set
print("\nTest Accuracy:", accuracy_score(y_test, y_test_pred))
print("\nTest F1 Score:", f1_score(y_test, y_test_pred, average='weighted'))
print("\nTest Classification Report:")
print(classification_report(y_test, y_test_pred, target_names=list(class_mapping.keys())))



Test Accuracy: 0.9106109324758842

Test F1 Score: 0.9090734132393893

Test Classification Report:
                        precision    recall  f1-score   support

             ALGORITHM       0.59      0.45      0.51        38
         COMPATIBILITY       0.65      0.61      0.63        18
                DEFECT       0.70      0.62      0.65        26
                DESIGN       0.86      0.88      0.87       436
         DOCUMENTATION       0.50      0.67      0.57         6
        IMPLEMENTATION       0.82      0.76      0.79        79
                  TEST       0.77      0.77      0.77        26
WITHOUT_CLASSIFICATION       0.97      0.98      0.97       926

              accuracy                           0.91      1555
             macro avg       0.73      0.72      0.72      1555
          weighted avg       0.91      0.91      0.91      1555

